In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

- Extracting Large Dataset
- Mccabe Metrics

In [ ]:
!pip install radon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 866.1 kB/s eta 0:00:00


In [ ]:
import os
import csv
from radon.complexity import cc_visit
from radon.metrics import h_visit,mi_visit
from radon.raw import analyze
import numpy as np
import torch
import textwrap
import pandas as pd

### Extracting Large Dataset

In [ ]:
!unrar x -o+ /content/drive/MyDrive/CSE_4554_ML_PROJECT/pytracebugs_dataset_v1.rar

Streaming output truncated to the last 5000 lines.
Extracting  buggy_dataset/buggy_snippets_files/e4af55411d01fc9ad265e91e62271413dcf0b7ae4c120b13743eec84adbf121e_after_merge.py       6%  OK 
Extracting  buggy_dataset/buggy_snippets_files/e4af55411d01fc9ad265e91e62271413dcf0b7ae4c120b13743eec84adbf121e_before_merge.py       6%  OK 
Extracting  buggy_dataset/buggy_snippets_files/e4b0da8c0e809745a6b593207b0dea52fa41e6fda4d5c80ea5984c0a312fd0d9_after_merge.py       6%  OK 
Extracting  buggy_dataset/buggy_snippets_files/e4b0da8c0e809745a6b593207b0dea52fa41e6fda4d5c80ea5984c0a312fd0d9_before_merge.py       6%  OK 
Extracting  buggy_dataset/buggy_snippets_files/e4b37a071376cfa71c7e181147802622a526d2ef5591dba008b2b9b0b5edd8f7_after_merge.py       6%  OK 
Extracting  buggy_dataset/buggy_snippets_files/e4b37a071376cfa71c7e181147802622a526d2ef5591dba008b2b9b0b5edd8f7_before_merge.py       6%  OK 
Extracting  buggy_dataset/buggy_snippets_files

In [ ]:
!tar -xf /content/stable_dataset/source_files.tar.gz

In [ ]:
!tar -xf /content/stable_dataset/stable_snippets_files.tar.gz

In [ ]:
import ast
import textwrap

def extract_attributes(node, attributes):
    if isinstance(node, ast.Attribute):
        attributes.append(node.attr)
        extract_attributes(node.value, attributes)
    elif isinstance(node, ast.Name):
        attributes.append(node.id)
    elif isinstance(node, ast.Subscript):
        extract_attributes(node.value, attributes)
    elif isinstance(node, ast.Call):
        extract_attributes(node.func, attributes)
        for arg in node.args:
            extract_attributes(arg, attributes)
        for keyword in node.keywords:
            extract_attributes(keyword.value, attributes)
    elif isinstance(node, ast.IfExp):
        extract_attributes(node.test, attributes)
        extract_attributes(node.body, attributes)
        extract_attributes(node.orelse, attributes)
    elif isinstance(node, ast.Expr):
        extract_attributes(node.value, attributes)
    elif isinstance(node, ast.List):
        for elt in node.elts:
            extract_attributes(elt, attributes)
    elif isinstance(node, ast.Dict):
        for key, value in zip(node.keys, node.values):
            extract_attributes(key, attributes)
            extract_attributes(value, attributes)

def calculate_ck_metrics(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        content = file.read()
    content = textwrap.dedent(content)
    try:
        tree = ast.parse(content)

        metrics = [0] * 3
        attributes = []

        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                metrics[0] += 1


            extract_attributes(node, attributes)


            if isinstance(node, (ast.If, ast.While, ast.For)):
                metrics[2] += 1


        metrics[1] = len(set(attributes))

    except Exception as e:
        print(f"error : {file_path}: {e}")
        metrics = []

    return metrics if metrics else []

In [ ]:
def process_files_for_ck_metrics(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".py"):
            file_path = os.path.join(folder_path, filename)
            metrics = calculate_ck_metrics(file_path)
            print([filename] + metrics)
            results.append([filename] + metrics)
    return results

In [ ]:
def save_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['filename', 'number of methods', 'number of attributes', 'number of branches'])
        for row in results:
            csv_writer.writerow(row)





Streaming output truncated to the last 5000 lines.
['76a8772f84d32189a198be9089e5f4eeca55c62ae40d6d126fa3e28f68173127_before_merge.py', 1, 56, 2]
['1a4ed238a8eeb44a2dd78b70254bbcfed9fedfe4df634eb6d2f5007baf867dda_after_merge.py', 1, 19, 19]
['976f76c140e1d1096eb010601ff6c3901ef6fff1c4c83bbf57a6aac177e3f48e_before_merge.py', 1, 7, 4]
['778953ae35755f92dbede963f6a19f9592ece722d7b775e7873b7039e21d32bc_after_merge.py', 1, 2, 0]
['612f8258c290eda5811487bd5aa8b2cc578f7a79f3fb8eeb5838487c9b070e41_after_merge.py', 1, 6, 11]
['6132bd702d45626e409fdc8a0cce8b5f9691ac3a4b6a922415c9b849a8e05e28_after_merge.py', 1, 8, 2]
['74be850fcd80c36bad0ba626a22e945d2ffd3d74f029286545d8fb6cbc8cf8b6_after_merge.py', 1, 1, 2]
['e731391da35f9cafba96acef80460075d7928c1ef4793c7c811aec8e09cecd67_before_merge.py', 1, 7, 18]
['89b68562366946b83b7ebc7b2793b37fbd89f5b6f63e129c8410fb509880d74a_before_merge.py', 1, 0, 0]
['c7c49469422618a6a4cf8714d85fca21a653952f6a792f7822bf7654fab1169c_after_merge.py', 1, 2, 14]
['1914c6a

In [ ]:
results_buggy_ck_ = process_files_for_ck_metrics('/content/buggy_dataset/buggy_snippets_files')
save_to_csv(results_buggy_ck_, '/content/drive/MyDrive/CSE_4554_ML_PROJECT/buggy_ck.csv')

In [ ]:
results_source_ck = process_files_for_ck_metrics('/content/source_files')
save_to_csv(results_source_ck,'/content/drive/MyDrive/CSE_4554_ML_PROJECT/source_ck.csv')

Streaming output truncated to the last 5000 lines.
['2483133c18dece411ce69da70d976d88a858b6fc.py', 25, 44, 48]
['00964bea08503115777572fc8794aeb3ab820956.py', 4, 13, 3]
['7a535d981c2fd076db053b31495a2e27a92bc894.py', 30, 26, 39]
['1d33657b94498edcd93c745538c617fabf278054.py', 24, 8, 0]
['0c038d894a07122899af0b97c3e18155a94045ac.py', 4, 16, 11]
['4b26474b7addedc81c07e14faef71115f0b47460.py', 3, 2, 4]
['ee0bc5786eda4462264c8e41f64e8097b0f41003.py', 5, 22, 22]
['c3439ff54fe78f86a23373da0ffe3193f21d2e6b.py', 8, 10, 11]
['8781126f3794619211b025cdbdf2ee7a2e0a3f62.py', 12, 6, 1]
['482cc4f47d8b8d0dcca00547e7c10c5371ad6158.py', 13, 10, 8]
['05c4c3bbb20ec9e83455d6f69915139017d00194.py', 2, 3, 1]
['1a956ffb1835b56bd7664cf7c6a68636034c641a.py', 10, 3, 5]
['b15640b28a83e6e7dea40611cfc4cc26f4f649cb.py', 3, 2, 2]
['95777c67a690dbf2f9810004acf9ce75e9d2742e.py', 1, 20, 13]
['76f9855fb7ce42b2a83738e9652f7e6c3f671fb9.py', 2, 1, 0]
['3fb539398adb1a06e27492911516688cdcc94d63.py', 7, 10, 9]
['ce6d2ad73422c5

In [ ]:
results_stable_ck = process_files_for_ck_metrics('/content/stable_snippets_files')
save_to_csv(results_stable_ck,'/content/drive/MyDrive/CSE_4554_ML_PROJECT/stable_ck.csv')

Streaming output truncated to the last 5000 lines.
['7e288ad386d5f7981dceb67ffeb190bdefe3aad0.py', 1, 6, 1]
['11597be1ecb3e50dbbacae4b19f1a274eddb4f15.py', 1, 0, 0]
['b72daf4dd09249093e465adf34c60ef34b8a0ada.py', 1, 8, 3]
['b64846ee89238d9e223c1b96a81b60d00f45f7db.py', 1, 3, 0]
['0472d25cb7eef05550e177c7bc73cf5c8eff0fb6.py', 1, 1, 0]
['5c10e8f8d379e91ae38b96034eeb7bc19ecd50da.py', 1, 0, 0]
['810d42da76d3f32b781de990032d5a0aeb8248e8.py', 1, 0, 0]
['840e7cf278d346586ef7a352ce79b42247341774.py', 1, 0, 0]
['435e03270f9df8ece9a7acc64999d54b488ae0f2.py', 1, 2, 1]
['6713f803a99475e8e0aa9319ee8df6f409852077.py', 1, 1, 1]
['d8aa9af7c49b59bb13078af935dd437ab7694f8e.py', 2, 10, 10]
['326d2e07425529c3ba0e8b624c990284f94983f1.py', 1, 1, 1]
['75f9fb863150b2626b15f99ec413ff1be259fd42.py', 1, 2, 1]
['65a52f6399b6297c75fad02db427ca72ec41eddc.py', 1, 0, 0]
['f3d62e37d5c1876fa3059a605d37d991ce63aa44.py', 1, 2, 6]
['96497d831c17668184c10ebeb3d2c9a03e384e6f.py', 1, 1, 1]
['4d851e9d6e09cb8471a8b8f506f1c32bc

In [ ]:
results_buggy_ck = process_files_for_ck_metrics('/content/buggy_dataset/buggy_snippets_files')
save_to_csv(results_buggy_ck, '/content/drive/MyDrive/buggy_ck_.csv')

Streaming output truncated to the last 5000 lines.
['76a8772f84d32189a198be9089e5f4eeca55c62ae40d6d126fa3e28f68173127_before_merge.py', 1, 63, 2]
['1a4ed238a8eeb44a2dd78b70254bbcfed9fedfe4df634eb6d2f5007baf867dda_after_merge.py', 1, 56, 19]
['976f76c140e1d1096eb010601ff6c3901ef6fff1c4c83bbf57a6aac177e3f48e_before_merge.py', 1, 24, 4]
['778953ae35755f92dbede963f6a19f9592ece722d7b775e7873b7039e21d32bc_after_merge.py', 1, 8, 0]
['612f8258c290eda5811487bd5aa8b2cc578f7a79f3fb8eeb5838487c9b070e41_after_merge.py', 1, 46, 11]
['6132bd702d45626e409fdc8a0cce8b5f9691ac3a4b6a922415c9b849a8e05e28_after_merge.py', 1, 31, 2]
['74be850fcd80c36bad0ba626a22e945d2ffd3d74f029286545d8fb6cbc8cf8b6_after_merge.py', 1, 17, 2]
['e731391da35f9cafba96acef80460075d7928c1ef4793c7c811aec8e09cecd67_before_merge.py', 1, 34, 18]
['89b68562366946b83b7ebc7b2793b37fbd89f5b6f63e129c8410fb509880d74a_before_merge.py', 1, 0, 0]
['c7c49469422618a6a4cf8714d85fca21a653952f6a792f7822bf7654fab1169c_after_merge.py', 1, 24, 14]
['1

In [ ]:
results_source_ck = process_files_for_ck_metrics('/content/source_files')
save_to_csv(results_source_ck,'/content/drive/MyDrive/source_ck_.csv')

Streaming output truncated to the last 5000 lines.
['2483133c18dece411ce69da70d976d88a858b6fc.py', 25, 144, 48]
['00964bea08503115777572fc8794aeb3ab820956.py', 4, 30, 3]
['7a535d981c2fd076db053b31495a2e27a92bc894.py', 30, 130, 39]
['1d33657b94498edcd93c745538c617fabf278054.py', 24, 60, 0]
['0c038d894a07122899af0b97c3e18155a94045ac.py', 4, 49, 11]
['4b26474b7addedc81c07e14faef71115f0b47460.py', 3, 8, 4]
['ee0bc5786eda4462264c8e41f64e8097b0f41003.py', 5, 68, 22]
['c3439ff54fe78f86a23373da0ffe3193f21d2e6b.py', 8, 54, 11]
['8781126f3794619211b025cdbdf2ee7a2e0a3f62.py', 12, 18, 1]
['482cc4f47d8b8d0dcca00547e7c10c5371ad6158.py', 13, 58, 8]
['05c4c3bbb20ec9e83455d6f69915139017d00194.py', 2, 23, 1]
['1a956ffb1835b56bd7664cf7c6a68636034c641a.py', 10, 34, 5]
['b15640b28a83e6e7dea40611cfc4cc26f4f649cb.py', 3, 33, 2]
['95777c67a690dbf2f9810004acf9ce75e9d2742e.py', 1, 69, 13]
['76f9855fb7ce42b2a83738e9652f7e6c3f671fb9.py', 2, 17, 0]
['3fb539398adb1a06e27492911516688cdcc94d63.py', 7, 54, 9]
['ce6d2a

In [ ]:
results_stable_ck_ = process_files_for_ck_metrics('/content/stable_snippets_files')
save_to_csv(results_stable_ck_,'/content/drive/MyDrive/stable_ck.csv')

Streaming output truncated to the last 5000 lines.
['7e288ad386d5f7981dceb67ffeb190bdefe3aad0.py', 1, 18, 1]
['11597be1ecb3e50dbbacae4b19f1a274eddb4f15.py', 1, 4, 0]
['b72daf4dd09249093e465adf34c60ef34b8a0ada.py', 1, 28, 3]
['b64846ee89238d9e223c1b96a81b60d00f45f7db.py', 1, 11, 0]
['0472d25cb7eef05550e177c7bc73cf5c8eff0fb6.py', 1, 6, 0]
['5c10e8f8d379e91ae38b96034eeb7bc19ecd50da.py', 1, 2, 0]
['810d42da76d3f32b781de990032d5a0aeb8248e8.py', 1, 7, 0]
['840e7cf278d346586ef7a352ce79b42247341774.py', 1, 5, 0]
['435e03270f9df8ece9a7acc64999d54b488ae0f2.py', 1, 15, 1]
['6713f803a99475e8e0aa9319ee8df6f409852077.py', 1, 9, 1]
['d8aa9af7c49b59bb13078af935dd437ab7694f8e.py', 2, 48, 10]
['326d2e07425529c3ba0e8b624c990284f94983f1.py', 1, 19, 1]
['75f9fb863150b2626b15f99ec413ff1be259fd42.py', 1, 8, 1]
['65a52f6399b6297c75fad02db427ca72ec41eddc.py', 1, 3, 0]
['f3d62e37d5c1876fa3059a605d37d991ce63aa44.py', 1, 21, 6]
['96497d831c17668184c10ebeb3d2c9a03e384e6f.py', 1, 9, 1]
['4d851e9d6e09cb8471a8b8f506f